In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cross_validation import cross_val_score
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [458]:
df_train = pd.read_csv("train.csv")

In [459]:
df_train.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [460]:
for i in df_train.columns:
    print(df_train[i].unique())

['LP001002' 'LP001003' 'LP001005' 'LP001006' 'LP001008' 'LP001011'
 'LP001013' 'LP001014' 'LP001018' 'LP001020' 'LP001024' 'LP001027'
 'LP001028' 'LP001029' 'LP001030' 'LP001032' 'LP001034' 'LP001036'
 'LP001038' 'LP001041' 'LP001043' 'LP001046' 'LP001047' 'LP001050'
 'LP001052' 'LP001066' 'LP001068' 'LP001073' 'LP001086' 'LP001087'
 'LP001091' 'LP001095' 'LP001097' 'LP001098' 'LP001100' 'LP001106'
 'LP001109' 'LP001112' 'LP001114' 'LP001116' 'LP001119' 'LP001120'
 'LP001123' 'LP001131' 'LP001136' 'LP001137' 'LP001138' 'LP001144'
 'LP001146' 'LP001151' 'LP001155' 'LP001157' 'LP001164' 'LP001179'
 'LP001186' 'LP001194' 'LP001195' 'LP001197' 'LP001198' 'LP001199'
 'LP001205' 'LP001206' 'LP001207' 'LP001213' 'LP001222' 'LP001225'
 'LP001228' 'LP001233' 'LP001238' 'LP001241' 'LP001243' 'LP001245'
 'LP001248' 'LP001250' 'LP001253' 'LP001255' 'LP001256' 'LP001259'
 'LP001263' 'LP001264' 'LP001265' 'LP001266' 'LP001267' 'LP001273'
 'LP001275' 'LP001279' 'LP001280' 'LP001282' 'LP001289' 'LP001

In [461]:
df_train['Married'][df_train['Married'].isnull()] = 'No'
df_train['Gender'][df_train['Gender'].isnull()] = df_train['Self_Employed'][df_train['Gender'].isnull()]
df_train['Gender'][df_train['Gender']=='Yes'] = 'Male'
df_train['Gender'][df_train['Gender']=='No'] = 'Female'
df_train['Dependents'][df_train['Dependents'].isnull()] = '0'
df_train['Self_Employed'][(df_train['Self_Employed'].isnull()) & (df_train['Education']=='Not Graduate')] = 'Yes'
df_train['Self_Employed'][(df_train['Self_Employed'].isnull()) & (df_train['Gender']=='Female')] = 'No'
df_train['Self_Employed'][(df_train['Self_Employed'].isnull()) & (df_train['ApplicantIncome']>5000)] = 'Yes'
df_train['Self_Employed'][(df_train['Self_Employed'].isnull()) & (df_train['ApplicantIncome']<=5000)] = 'No'
df_train['LoanAmount'][df_train['LoanAmount'].isnull()] = df_train['LoanAmount'].median()
df_train['Loan_Amount_Term'][df_train['Loan_Amount_Term'].isnull()] = df_train['Loan_Amount_Term'].median()
df_train['Credit_History'][df_train['Credit_History'].isnull()] = 1.0

In [462]:
df_train.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [463]:
df_train.corr(method='pearson')

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
ApplicantIncome,1.000000,-0.116605,0.565181,-0.046531,-0.018615
CoapplicantIncome,-0.116605,1.000000,0.189218,-0.059383,0.011134
LoanAmount,0.565181,0.189218,1.000000,0.036960,-0.000607
Loan_Amount_Term,-0.046531,-0.059383,0.036960,1.000000,-0.004705
Credit_History,-0.018615,0.011134,-0.000607,-0.004705,1.000000


In [464]:
df_train['Loan_Status'][df_train['Loan_Status']=='Y'] = 1
df_train['Loan_Status'][df_train['Loan_Status']=='N'] = 0

In [465]:
df_y_train = df_train['Loan_Status']
del df_train['Loan_Status']
del df_train['Loan_ID']


In [466]:
df_x_train = pd.get_dummies(df_train)

In [467]:
df_x_train.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Female', 'Gender_Male',
       'Married_No', 'Married_Yes', 'Dependents_0', 'Dependents_1',
       'Dependents_2', 'Dependents_3+', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban',
       'Property_Area_Urban'],
      dtype='object')

In [468]:
df_x_train.shape

(614, 20)

In [479]:
from sklearn.feature_selection import RFE
rfe = RFE(rfc,10)

In [480]:
df_y_train=df_y_train.astype('int')
filtered_columns = rfe.fit(df_x_train,df_y_train)

In [481]:
df_x_train_rfe = df_x_train[df_x_train.columns[filtered_columns.support_]]

In [482]:
df_x_train_rfe.head(2)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Married_No,Married_Yes,Education_Graduate,Property_Area_Rural,Property_Area_Semiurban
0,5849,0.0,128.0,360.0,1.0,1,0,1,0,0
1,4583,1508.0,128.0,360.0,1.0,0,1,1,1,0


In [483]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth = 5,criterion='entropy',n_estimators=300)

In [484]:
X_train,X_test,Y_train,Y_test = train_test_split(df_x_train,df_y_train,test_size=0.25,random_state=5)

In [485]:
rfc.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [486]:
rfc.score(X_test,Y_test)

0.8116883116883117

In [487]:
cross_val_score(rfc_pca,df_x_train_rfe,df_y_train,cv=15).mean()

0.8062717770034844

In [488]:
from sklearn.svm import SVC
svc_model = SVC(kernel = "linear", C=2,gamma=1)
svc_model.fit(X_train,Y_train)
svc_predict = svc_model.predict(X_test)
print (accuracy_score(Y_test,svc_predict))
print (confusion_matrix(Y_test,svc_predict))
print (classification_report(Y_test,svc_predict))

0.7922077922077922
[[ 18  28]
 [  4 104]]
             precision    recall  f1-score   support

          0       0.82      0.39      0.53        46
          1       0.79      0.96      0.87       108

avg / total       0.80      0.79      0.77       154



In [261]:
df_test1 = pd.read_csv("test.csv")

In [262]:
df_test

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban
5,LP001054,Male,Yes,0,Not Graduate,Yes,2165,3422,152.0,360.0,1.0,Urban
6,LP001055,Female,No,1,Not Graduate,No,2226,0,59.0,360.0,1.0,Semiurban
7,LP001056,Male,Yes,2,Not Graduate,No,3881,0,147.0,360.0,0.0,Rural
8,LP001059,Male,Yes,2,Graduate,NaN,13633,0,280.0,240.0,1.0,Urban
9,LP001067,Male,No,0,Not Graduate,No,2400,2400,123.0,360.0,1.0,Semiurban


In [263]:
for i in df_test.columns:
    print(df_test[i].unique())

['LP001015' 'LP001022' 'LP001031' 'LP001035' 'LP001051' 'LP001054'
 'LP001055' 'LP001056' 'LP001059' 'LP001067' 'LP001078' 'LP001082'
 'LP001083' 'LP001094' 'LP001096' 'LP001099' 'LP001105' 'LP001107'
 'LP001108' 'LP001115' 'LP001121' 'LP001124' 'LP001128' 'LP001135'
 'LP001149' 'LP001153' 'LP001163' 'LP001169' 'LP001174' 'LP001176'
 'LP001177' 'LP001183' 'LP001185' 'LP001187' 'LP001190' 'LP001203'
 'LP001208' 'LP001210' 'LP001211' 'LP001219' 'LP001220' 'LP001221'
 'LP001226' 'LP001230' 'LP001231' 'LP001232' 'LP001237' 'LP001242'
 'LP001268' 'LP001270' 'LP001284' 'LP001287' 'LP001291' 'LP001298'
 'LP001312' 'LP001313' 'LP001317' 'LP001321' 'LP001323' 'LP001324'
 'LP001332' 'LP001335' 'LP001338' 'LP001347' 'LP001348' 'LP001351'
 'LP001352' 'LP001358' 'LP001359' 'LP001361' 'LP001366' 'LP001368'
 'LP001375' 'LP001380' 'LP001386' 'LP001400' 'LP001407' 'LP001413'
 'LP001415' 'LP001419' 'LP001420' 'LP001428' 'LP001445' 'LP001446'
 'LP001450' 'LP001452' 'LP001455' 'LP001466' 'LP001471' 'LP001

In [264]:
df_test['Married'][df_test['Married'].isnull()] = 'No'
df_test['Gender'][df_test['Gender'].isnull()] = df_test['Self_Employed'][df_test['Gender'].isnull()]
df_test['Gender'][df_test['Gender']=='Yes'] = 'Male'
df_test['Gender'][df_test['Gender']=='No'] = 'Female'
df_test['Dependents'][df_test['Dependents'].isnull()] = '0'
df_test['Self_Employed'][(df_test['Self_Employed'].isnull()) & (df_test['Education']=='Not Graduate')] = 'Yes'
df_test['Self_Employed'][(df_test['Self_Employed'].isnull()) & (df_test['Gender']=='Female')] = 'No'
df_test['Self_Employed'][(df_test['Self_Employed'].isnull()) & (df_test['ApplicantIncome']>5000)] = 'Yes'
df_test['Self_Employed'][(df_test['Self_Employed'].isnull()) & (df_test['ApplicantIncome']<=5000)] = 'No'
df_test['LoanAmount'][df_test['LoanAmount'].isnull()] = df_test['LoanAmount'].median()
df_test['Loan_Amount_Term'][df_test['Loan_Amount_Term'].isnull()] = df_test['Loan_Amount_Term'].median()
df_test['Credit_History'][df_test['Credit_History'].isnull()] = 1.0

In [265]:
for i in df_test.columns:
    print(df_test[i].unique())

['LP001015' 'LP001022' 'LP001031' 'LP001035' 'LP001051' 'LP001054'
 'LP001055' 'LP001056' 'LP001059' 'LP001067' 'LP001078' 'LP001082'
 'LP001083' 'LP001094' 'LP001096' 'LP001099' 'LP001105' 'LP001107'
 'LP001108' 'LP001115' 'LP001121' 'LP001124' 'LP001128' 'LP001135'
 'LP001149' 'LP001153' 'LP001163' 'LP001169' 'LP001174' 'LP001176'
 'LP001177' 'LP001183' 'LP001185' 'LP001187' 'LP001190' 'LP001203'
 'LP001208' 'LP001210' 'LP001211' 'LP001219' 'LP001220' 'LP001221'
 'LP001226' 'LP001230' 'LP001231' 'LP001232' 'LP001237' 'LP001242'
 'LP001268' 'LP001270' 'LP001284' 'LP001287' 'LP001291' 'LP001298'
 'LP001312' 'LP001313' 'LP001317' 'LP001321' 'LP001323' 'LP001324'
 'LP001332' 'LP001335' 'LP001338' 'LP001347' 'LP001348' 'LP001351'
 'LP001352' 'LP001358' 'LP001359' 'LP001361' 'LP001366' 'LP001368'
 'LP001375' 'LP001380' 'LP001386' 'LP001400' 'LP001407' 'LP001413'
 'LP001415' 'LP001419' 'LP001420' 'LP001428' 'LP001445' 'LP001446'
 'LP001450' 'LP001452' 'LP001455' 'LP001466' 'LP001471' 'LP001

In [266]:
df_test = pd.get_dummies(df_test)

In [271]:
df_test

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_ID_LP001015',
       'Loan_ID_LP001022', 'Loan_ID_LP001031', 'Loan_ID_LP001035',
       'Loan_ID_LP001051',
       ...
       'Dependents_1', 'Dependents_2', 'Dependents_3+', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban',
       'Property_Area_Urban'],
      dtype='object', length=387)

In [275]:
df_test_final = df_test[df_x_train.columns[filtered_columns.support_]]

In [438]:
y_pred = rfc.predict(df_test_final)

In [439]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [440]:
final = pd.read_csv("Sample_Submission.csv")

In [441]:
final['Loan_ID'] = df_test1['Loan_ID']

In [442]:
df_test1 = pd.read_csv("test.csv")

In [443]:
final['Loan_Status'] = y_pred

In [446]:
final['Loan_Status'][final['Loan_Status']==1]='Y'
final['Loan_Status'][final['Loan_Status']==0]='N'

In [447]:
import xlsxwriter
import pandas as pd
# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('Sample_Submission.xlsx')
worksheet = workbook.add_worksheet()
workbook.close()
writer = pd.ExcelWriter('Sample_Submission.xlsx')
final.to_excel(writer,'Sheet1')
writer.save()